In [1]:
import polars as pl
import pysentiment2 as ps
import math
import os
import re
import urllib.parse

In [2]:
df_dict = {}

for company_name in os.listdir(r"C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred"):
    
    file_path = os.path.join(r"C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred", company_name)

    match = re.search(r"\\([A-Z]+(?:\.[A-Z])?)_reports\.parquet", file_path)

    if match:
        ticker_symbol = match.group(1)
        print("Ticker Symbol:", ticker_symbol, '\n', file_path)
    else:
        print(f"Ticker symbol for {company_name} is not found")

    df_name = f"{ticker_symbol}"
    df = pl.read_parquet(file_path)

    df_dict[df_name] = df

Ticker Symbol: AAL 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AAL_reports.parquet
Ticker Symbol: AAPL 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AAPL_reports.parquet
Ticker Symbol: ABBV 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ABBV_reports.parquet
Ticker Symbol: ABNB 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ABNB_reports.parquet
Ticker Symbol: ABT 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ABT_reports.parquet
Ticker Symbol: ACGL 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ACGL_reports.parquet
Ticker Symbol: ACN 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ACN_reports.parquet
Ticker Symbol: ADBE 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ADBE_reports.parquet
Ticker Symbol: ADI 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ADI_reports.parquet


In [3]:
print(f'Total number of companies: {len(df_dict)}')

Total number of companies: 497


In [4]:
total_num_of_rep = 0

for value in df_dict.values():
    total_num_of_rep += value.width
print(total_num_of_rep)

9830


In [5]:
total_num_of_words = 0

for value in df_dict.values():
    for col in value:
        total_num_of_words += col.count()

print(total_num_of_words)

264502259


In [6]:
#average doc length
average_word_count = total_num_of_words/total_num_of_rep
print(f'Average doc length in words: {round(average_word_count)}')
ln_average_word_count = math.log(average_word_count)

Average doc length in words: 26908


In [7]:
#here I have checked if there ara weird numder of words for reports
for company_name, df in df_dict.items():
    for col in df:
        if col.count() < 5000:
            print(f"{company_name}, {col.name}: {col.count()}")
        elif col.count() > 100000:
            print(f"{company_name}, {col.name}: {col.count()}")

AAL, 2022-02-22: 103357
AEP, 2021-02-25: 114290
AEP, 2022-02-24: 116697
AEP, 2023-02-23: 124904
AIG, 2019-02-15: 126596
AIG, 2020-02-21: 127540
AIG, 2021-02-19: 141344
AIG, 2022-02-17: 141905
AIG, 2023-02-17: 117387
AMCR, 2019-05-09: 3541
BAC, 2021-02-24: 101063
BK, 2019-02-27: 4573
BK, 2020-02-27: 4046
BK, 2022-02-25: 4911
BX, 2019-03-01: 108861
BX, 2020-02-28: 101887
BX, 2021-02-26: 105295
BX, 2022-02-25: 104668
BX, 2023-02-24: 104516
CVS, 2023-02-08: 102008
C, 2021-02-26: 127501
C, 2022-02-28: 128374
C, 2023-02-27: 131435
DUK, 2021-02-25: 100697
EMR, 2020-02-05: 4067
EMR, 2021-02-03: 4342
ETR, 2019-02-26: 168747
ETR, 2020-02-21: 160811
ETR, 2021-02-26: 195291
ETR, 2022-02-25: 199557
ETR, 2023-02-24: 223235
EXC, 2019-02-08: 133056
EXC, 2021-02-24: 117172
EXC, 2022-02-25: 105179
FITB, 2021-02-26: 114400
FITB, 2022-02-25: 108102
FITB, 2023-02-24: 101084
GS, 2021-02-22: 106729
GS, 2022-02-25: 109072
GS, 2023-02-24: 131119
HIG, 2021-02-19: 110565
HIG, 2022-02-18: 105016
JPM, 2019-02-26: 

This is the function that takes polars series as an input, then concatinate all values in the single string. For this sitring, which is the initial report, this func calculates sentiment scores according to Harvard IV-4 and Loughran and McDonald Financial Sentiment Dictionaries. Particulary, this function calculates raw scores which is simple difference between positive and negative words count and polarity score which is defined by the following formula:

$$
Polarity = \frac{N_{\text{pos}} - N_{\text{neg}}}{N_{\text{pos}} + N_{\text{neg}}}
$$

Also this func returns simple word count (document length) to use it as a control variable in regression.

In [8]:
lm = ps.LM()
hiv4 = ps.HIV4() 

def compute_pysentiment(series):

    score_lst = []

    doc_string = series.str.concat(" ", ignore_nulls=True)

    tokens_lm = lm.tokenize(doc_string[0])
    tokens_hiv4 = hiv4.tokenize(doc_string[0])

    score_lm = lm.get_score(tokens_lm)
    score_hiv4 = hiv4.get_score(tokens_hiv4)

    raw_lm = score_lm['Positive'] - score_lm['Negative']
    raw_hiv4 = score_hiv4['Positive'] - score_hiv4['Negative']

    score_lst.append(score_lm['Polarity'])
    score_lst.append(raw_lm)

    score_lst.append(score_hiv4['Polarity'])
    score_lst.append(raw_hiv4)

    score_lst.append(series.count())
  
    return score_lst

In [9]:
#example df and series 
example_series = df_dict['ALGN'][df_dict['ALGN'].columns[1]]
example_df = df_dict['ALGN']
example_df

2019-02-28,2019-05-02,2019-08-01,2019-10-31,2020-02-28,2020-05-05,2020-07-31,2020-10-30,2021-02-26,2021-05-05,2021-08-04,2021-11-02,2022-02-25,2022-05-05,2022-08-04,2022-11-04,2023-02-27,2023-05-05,2023-08-04,2023-11-03
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED"""
"""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…"
"""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND"""
"""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE"""
"""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,null,null,null,null,"""capacities""",null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,"""and""",null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,"""on""",null,null,null,null,null,null,null,null,null,null,null


In [10]:
#the resulting list contain 1: lm_polarity score; 2: lm_raw score; 3: hiv4_polarity score; 4: hiv4_raw score; 5: doc_length.
compute_pysentiment(example_series)

[-0.44518272388273855, -536, 0.3756441550377966, 1385, 23700]

In [13]:
for company_name, df in df_dict.items():

    score_dict = {}

    for col in df:
        
        score_lst = compute_pysentiment(col)

        score_dict[col.name] = score_lst
    
    df = pl.DataFrame(score_dict)
    print(df)
    
   # print(f'Iteration for {company_name} is done successfully')
#
 # #Determine the output directory and file name
   # output_dir = os.path.join('data', 'snp_500_scores')
   # os.makedirs(output_dir, exist_ok=True)
#
   # file_name_new = f"{company_name}.parquet"
   # full_path = os.path.join(output_dir, file_name_new)
   # full_path = os.path.normpath(full_path)
#
   # print(f"Attempting to write to: {full_path}")
#
   # # Write the DataFrame to Parquet
   # df.write_parquet(full_path)
#
   # print(f"{full_path} created successfully")

shape: (5, 20)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ 2019-02-2 ┆ 2019-04-2 ┆ 2019-07-2 ┆ 2019-10-2 ┆ … ┆ 2023-02-2 ┆ 2023-04-2 ┆ 2023-07-2 ┆ 2023-10- │
│ 5         ┆ 6         ┆ 5         ┆ 4         ┆   ┆ 2         ┆ 7         ┆ 0         ┆ 19       │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ -0.309005 ┆ -0.427839 ┆ -0.437649 ┆ -0.426772 ┆ … ┆ -0.358016 ┆ -0.486272 ┆ -0.465144 ┆ -0.45454 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 5        │
│ -652.0    ┆ -501.0    ┆ -551.0    ┆ -542.0    ┆ … ┆ -1025.0   ┆ -797.0    ┆ -774.0    ┆ -745.0   │
│ 0.479639  ┆ 0.367517  ┆ 0.362751  ┆ 0.373458  ┆ … ┆ 0.509044  ┆ 0.390974  

KeyboardInterrupt: 

jupyter nbconvert --to html Parsim-sec\src\Analysis\snp_500_scores.ipynb